In [ ]:
import json
import os
import pandas
from pprint import pprint

In [ ]:
dirs = ['biorxiv_medrxiv']

for d in dirs:
    files = os.listdir(d)
    for file in files:
        filepath = os.path.join(d, file)
        with open(filepath, 'rb') as f:
            file_data = json.load(f)
            title = file_data['metadata']['title']
            abstract_data = file_data['abstract']  # List of paragraphs in abstract
            # TODO: Concatenate paragraphs
            body_text = file_data['body_text']